In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm.models as models
import numpy as np
import pandas as pd
from utils import US8K
def train_synthetic(model,root="./synthetic_us8k",transform_size=128,train=True):
    loss_fn=nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer=torch.optim.AdamW(model.parameters(),lr=0.0001)
    train_dataset=US8K(train=True,transform_size=128)
    print(len(train_dataset.data))
    synthetic_dataset=US8K(train=True,root=root,transform_size=transform_size)
    print(len(synthetic_dataset.data))
    test_dataset=US8K(train=False,transform_size=128)
    epoches=500
    print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    for epoch in range(epoches):
        model.train()
        if(epoch%2<=0):
            train_loader=torch.utils.data.DataLoader(synthetic_dataset,batch_size=100,shuffle=True)
        else:
            train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=100,shuffle=True)
        test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=100,shuffle=True)
        accuracy=0
        losses_all=0
        all_have=0
        for data,label in train_loader:
            data=data.cuda()
            label=label.cuda()
            output=model(data)
            loss=loss_fn(output,label)
            accuracy+=torch.sum(torch.argmax(output,dim=1)==torch.argmax(label,dim=1))
            all_have+=data.shape[0]
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses_all+=loss.item()
        print('epoch:{},train_loss:{},train_accuracy:{}'.format(epoch,losses_all/all_have,accuracy/all_have)) 

        model.eval()
        all_have=0
    
        accuracy=0
        losses_all=0
        for data,label in test_loader:
            data=data.cuda()
            label=label.cuda()
            output=model(data)
            loss=loss_fn(output,label)
            all_have+=data.shape[0]
            accuracy+=torch.sum(torch.argmax(output,dim=1)==torch.argmax(label,dim=1))
            losses_all+=loss.item()
        print('epoch:{},test_loss:{},test_accuracy:{}'.format(epoch,losses_all/all_have,accuracy/all_have))
def train(model):
    loss_fn=nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer=torch.optim.AdamW(model.parameters(),lr=0.0001)
    train_dataset=US8K(train=True,transform_size=128,)
    print(len(train_dataset.data))
    synthetic_dataset=US8K(train=True,transform_size=128)
    print(len(synthetic_dataset.data))
    test_dataset=US8K(train=False,transform_size=128)
    epoches=500
    print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    for epoch in range(epoches):
        model.train()
        if(epoch%2<=0):
            train_loader=torch.utils.data.DataLoader(synthetic_dataset,batch_size=100,shuffle=True)
        else:
            train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=100,shuffle=True)
        test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=100,shuffle=True)
        accuracy=0
        losses_all=0
        all_have=0
        for data,label in train_loader:
            data=data.cuda()
            label=label.cuda()
            output=model(data)
            loss=loss_fn(output,label)
            accuracy+=torch.sum(torch.argmax(output,dim=1)==torch.argmax(label,dim=1))
            all_have+=data.shape[0]
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses_all+=loss.item()
        print('epoch:{},train_loss:{},train_accuracy:{}'.format(epoch,losses_all/all_have,accuracy/all_have)) 

        model.eval()
        all_have=0
    
        accuracy=0
        losses_all=0
        for data,label in test_loader:
            data=data.cuda()
            label=label.cuda()
            output=model(data)
            loss=loss_fn(output,label)
            all_have+=data.shape[0]
            accuracy+=torch.sum(torch.argmax(output,dim=1)==torch.argmax(label,dim=1))
            losses_all+=loss.item()
        print('epoch:{},test_loss:{},test_accuracy:{}'.format(epoch,losses_all/all_have,accuracy/all_have))

In [4]:
#ResNet  Simple Augmentation  0.73
from utils import ESC,US8K
# from timm.models.convnext import convnext_tiny
from timm.models.resnet import resnet50
model=resnet50(num_classes=10,pretrained=False).cuda()

train(model)
#ResNet  #Augmentation 
#Augmentation  2000 0.726  48000 0.764

# from timm.models.convnext import convnext_tiny


100%|██████████| 8732/8732 [00:33<00:00, 257.85it/s]


7859 0
7859


100%|██████████| 8732/8732 [00:27<00:00, 314.81it/s]


7859 0
7859


100%|██████████| 8732/8732 [00:21<00:00, 409.79it/s]


0 873
23528522
epoch:0,train_loss:0.01816217587458024,train_accuracy:0.40946683287620544
epoch:0,test_loss:0.017025161718038622,test_accuracy:0.520045816898346
epoch:1,train_loss:0.014434868771545395,train_accuracy:0.6164906024932861
epoch:1,test_loss:0.01582562855155744,test_accuracy:0.5074456334114075
epoch:2,train_loss:0.011233519955678877,train_accuracy:0.7618017196655273
epoch:2,test_loss:0.016143927596005924,test_accuracy:0.5486827492713928
epoch:3,train_loss:0.009260129179264665,train_accuracy:0.8501080870628357
epoch:3,test_loss:0.013933824646240794,test_accuracy:0.639175295829773
epoch:4,train_loss:0.007994883584375523,train_accuracy:0.9094032049179077
epoch:4,test_loss:0.016852886679519355,test_accuracy:0.5246277451515198
epoch:5,train_loss:0.007093384772831133,train_accuracy:0.9488484263420105
epoch:5,test_loss:0.0132701559438192,test_accuracy:0.6449026465415955
epoch:6,train_loss:0.006535159121762252,train_accuracy:0.9740424752235413
epoch:6,test_loss:0.014423949338862712,t

In [7]:
#ResNet  #Synthetic  0.8 
from utils import US8K
# from timm.models.convnext import convnext_tiny
from timm.models.resnet import resnet50
model=resnet50(num_classes=10,pretrained=False).cuda()
train_synthetic(model,root="./Synthetic_us8k_2") #0.77
train_synthetic(model,root="./Synthetic_us8k_3")
train_synthetic(model,root="./Synthetic_us8k_4")
train_synthetic(model,root="./Synthetic_us8k_5")


100%|██████████| 8732/8732 [00:15<00:00, 562.04it/s]


7859 0
7859


100%|██████████| 9061/9061 [00:26<00:00, 344.62it/s]


9061 0
9061


100%|██████████| 8732/8732 [00:12<00:00, 710.83it/s]


...

100%|██████████| 8756/8756 [00:25<00:00, 347.53it/s]


8756 0
8756


100%|██████████| 8732/8732 [00:13<00:00, 639.64it/s]


0 873
23528522
epoch:0,train_loss:0.007188410935144873,train_accuracy:0.9207400679588318
epoch:0,test_loss:0.014372533266213888,test_accuracy:0.6781214475631714
epoch:1,train_loss:0.005067073740657618,train_accuracy:0.9999999403953552
epoch:1,test_loss:0.013725572025653013,test_accuracy:0.7273768782615662
epoch:2,train_loss:0.0062453907856172275,train_accuracy:0.9530608057975769
epoch:2,test_loss:0.01439868652806932,test_accuracy:0.6827033758163452
epoch:3,train_loss:0.005044018733833625,train_accuracy:0.9999999403953552
epoch:3,test_loss:0.013427464276518062,test_accuracy:0.7239404916763306
epoch:4,train_loss:0.005720890837208002,train_accuracy:0.9753312468528748
epoch:4,test_loss:0.014353758963795735,test_accuracy:0.6907216906547546
epoch:5,train_loss:0.0050377916221871055,train_accuracy:0.9999999403953552
epoch:5,test_loss:0.013709515776967402,test_accuracy:0.7308133244514465
epoch:6,train_loss:0.005458352655893921,train_accuracy:0.9845820069313049
epoch:6,test_loss:0.01673533116282

KeyboardInterrupt: 

In [8]:
#xception Augmentation:  0.72
from timm.models.convnext import convnext_base
from timm.models.xception import xception,Xception
model=xception(num_classes=10,pretrained=False).cuda()
train(model)

100%|██████████| 8732/8732 [00:19<00:00, 454.44it/s]


7859 0
7859


100%|██████████| 8731/8731 [00:19<00:00, 450.23it/s]


7858 0
7858


100%|██████████| 8732/8732 [00:12<00:00, 710.81it/s]


0 873
20827442
epoch:0,train_loss:0.018063980573065318,train_accuracy:0.40761005878448486
epoch:0,test_loss:0.018102118103228346,test_accuracy:0.44788089394569397
epoch:1,train_loss:0.011509278626338203,train_accuracy:0.734699010848999
epoch:1,test_loss:0.013685381153058623,test_accuracy:0.6506300568580627
epoch:2,train_loss:0.012726686691989059,train_accuracy:0.6653092503547668
epoch:2,test_loss:0.01648726591676098,test_accuracy:0.5085911154747009
epoch:3,train_loss:0.008198087111367632,train_accuracy:0.8821732401847839
epoch:3,test_loss:0.01321132308702152,test_accuracy:0.6884307265281677
epoch:4,train_loss:0.00990698069189911,train_accuracy:0.8089844584465027
epoch:4,test_loss:0.019701801078289508,test_accuracy:0.4501718282699585
epoch:5,train_loss:0.00686635782524805,train_accuracy:0.9507570266723633
epoch:5,test_loss:0.013027669228229326,test_accuracy:0.6792669296264648
epoch:6,train_loss:0.007784667566369344,train_accuracy:0.9149910807609558
epoch:6,test_loss:0.01828561369905767,

In [3]:
#xception Synthetic   0.80
from timm.models.xception import xception
model=xception(num_classes=10,pretrained=False).cuda()
train_synthetic(model)

100%|██████████| 8732/8732 [00:28<00:00, 310.86it/s]


7859 0
7859


100%|██████████| 8493/8493 [00:16<00:00, 513.87it/s]


8493 0
8493


100%|██████████| 8732/8732 [00:15<00:00, 563.29it/s]


0 873
20827442
epoch:0,train_loss:0.011060611562763691,train_accuracy:0.7536795139312744
epoch:0,test_loss:0.017135743286464906,test_accuracy:0.506300151348114
epoch:1,train_loss:0.010764940979675204,train_accuracy:0.7727445960044861
epoch:1,test_loss:0.01504627358326251,test_accuracy:0.5956472158432007
epoch:2,train_loss:0.006797306335226436,train_accuracy:0.9434828758239746
epoch:2,test_loss:0.01343704444299722,test_accuracy:0.6494845747947693
epoch:3,train_loss:0.007275888651044394,train_accuracy:0.9281078577041626
epoch:3,test_loss:0.014771280010131626,test_accuracy:0.6414662599563599
epoch:4,train_loss:0.006030452710417461,train_accuracy:0.976333498954773
epoch:4,test_loss:0.013580830919237345,test_accuracy:0.6689576506614685
epoch:5,train_loss:0.00607462665903946,train_accuracy:0.9804045557975769
epoch:5,test_loss:0.013995176601519154,test_accuracy:0.6517755389213562
epoch:6,train_loss:0.005588700377476534,train_accuracy:0.9919934272766113
epoch:6,test_loss:0.01217559884640496,te

In [5]:
#xception Synthetic  k varies 0.80
from timm.models.xception import xception
model=xception(num_classes=10,pretrained=False).cuda()
train_synthetic(model,root="./Synthetic_us8k_2") #0.787
# train_synthetic(model,root="./Synthetic_us8k_3") #0.748
# train_synthetic(model,root="./Synthetic_us8k_4") #0.745
# train_synthetic(model,root="./Synthetic_us8k_5") #0.740


100%|██████████| 8732/8732 [00:55<00:00, 156.08it/s]


7859 0
7859


100%|██████████| 9061/9061 [00:27<00:00, 330.98it/s]


9061 0
9061


100%|██████████| 8732/8732 [00:20<00:00, 431.72it/s]


0 873
20827442
epoch:0,train_loss:0.012416350579711726,train_accuracy:0.6848030090332031
epoch:0,test_loss:0.016035057561640614,test_accuracy:0.5028637051582336
epoch:1,train_loss:0.010881311119262582,train_accuracy:0.763455867767334
epoch:1,test_loss:0.01359512429057118,test_accuracy:0.6208477020263672
epoch:2,train_loss:0.007675540873499425,train_accuracy:0.9001213908195496
epoch:2,test_loss:0.014839417336323006,test_accuracy:0.6231386065483093
epoch:3,train_loss:0.007558752788449535,train_accuracy:0.9124569892883301
epoch:3,test_loss:0.013861517452838631,test_accuracy:0.6540664434432983
epoch:4,train_loss:0.006665129276606542,train_accuracy:0.948901891708374
epoch:4,test_loss:0.015059749695033962,test_accuracy:0.612829327583313
epoch:5,train_loss:0.006321251157252657,train_accuracy:0.9704796671867371
epoch:5,test_loss:0.013609169138529319,test_accuracy:0.694158136844635
epoch:6,train_loss:0.005866201504639029,train_accuracy:0.9834455251693726
epoch:6,test_loss:0.0127840200513772,tes

In [11]:
#convit_tiny Augmentation  0.65
from timm.models.convit import convit_base,convit_tiny,ConViT
model=convit_tiny(img_size=(128,128),num_classes=10,pretrained=False).cuda()
train(model)

100%|██████████| 8732/8732 [00:19<00:00, 456.61it/s]


7859 0
7859


100%|██████████| 8731/8731 [00:18<00:00, 473.69it/s]


7858 0
7858


100%|██████████| 8732/8732 [00:12<00:00, 694.26it/s]


0 873
5494098
epoch:0,train_loss:0.021949774650375665,train_accuracy:0.18847034871578217
epoch:0,test_loss:0.022131519765515233,test_accuracy:0.23024055361747742
epoch:1,train_loss:0.020508987422146588,train_accuracy:0.24608728289604187
epoch:1,test_loss:0.019872390117841896,test_accuracy:0.3505154848098755
epoch:2,train_loss:0.021319463769422897,train_accuracy:0.2206668257713318
epoch:2,test_loss:0.020256931566017735,test_accuracy:0.37457045912742615
epoch:3,train_loss:0.018775172384659863,train_accuracy:0.3426644504070282
epoch:3,test_loss:0.01878890931128363,test_accuracy:0.35967928171157837
epoch:4,train_loss:0.01991925216200516,train_accuracy:0.29969456791877747
epoch:4,test_loss:0.019562197305082865,test_accuracy:0.3665521442890167
epoch:5,train_loss:0.01719866532802521,train_accuracy:0.4368239939212799
epoch:5,test_loss:0.017996967180214932,test_accuracy:0.4410080313682556
epoch:6,train_loss:0.018739900142371608,train_accuracy:0.3671417534351349
epoch:6,test_loss:0.0183566651666

In [12]:
#convit_tiny Synthetic  0.686
from timm.models.convit import convit_base
model=convit_tiny(img_size=(128,128),num_classes=10,pretrained=False).cuda()
train_synthetic(model)

100%|██████████| 8732/8732 [00:19<00:00, 445.96it/s]


7859 0
7859


100%|██████████| 8493/8493 [00:09<00:00, 863.82it/s]


8493 0
8493


100%|██████████| 8732/8732 [00:14<00:00, 608.54it/s]


0 873
5494098
epoch:0,train_loss:0.019312310187931437,train_accuracy:0.3061344623565674
epoch:0,test_loss:0.022456622478751655,test_accuracy:0.2966781258583069
epoch:1,train_loss:0.01964098842320877,train_accuracy:0.303982675075531
epoch:1,test_loss:0.01888138881936789,test_accuracy:0.3814433217048645
epoch:2,train_loss:0.01590701996978343,train_accuracy:0.47062286734580994
epoch:2,test_loss:0.020949042539825963,test_accuracy:0.3470790684223175
epoch:3,train_loss:0.01809043242042795,train_accuracy:0.3762564957141876
epoch:3,test_loss:0.01829289843809427,test_accuracy:0.42611685395240784
epoch:4,train_loss:0.013837225681674588,train_accuracy:0.5924879312515259
epoch:4,test_loss:0.019241115208480503,test_accuracy:0.4421535134315491
epoch:5,train_loss:0.016876920031930885,train_accuracy:0.44878479838371277
epoch:5,test_loss:0.016846227755115066,test_accuracy:0.478808730840683
epoch:6,train_loss:0.012353734850504507,train_accuracy:0.6651360392570496
epoch:6,test_loss:0.018266934709450632,t

In [7]:
#convit_tiny Synthetic2  0.686
from timm.models.convit import convit_base,convit_tiny
model=convit_tiny(img_size=(128,128),num_classes=10,pretrained=False).cuda()
train_synthetic(model,root="./Synthetic_us8k_2")  #0.687
train_synthetic(model,root="./Synthetic_us8k_3") #0.645
train_synthetic(model,root="./Synthetic_us8k_4") #0.646
train_synthetic(model,root="./Synthetic_us8k_5") #0.647

100%|██████████| 8732/8732 [00:16<00:00, 542.60it/s]


7859 0
7859


100%|██████████| 9061/9061 [00:07<00:00, 1227.36it/s]


9061 0
9061


100%|██████████| 8732/8732 [00:12<00:00, 682.42it/s]


...

7859 0
7859


100%|██████████| 8756/8756 [00:27<00:00, 321.55it/s]


8756 0
8756


100%|██████████| 8732/8732 [00:22<00:00, 383.46it/s]


0 873
5494098
epoch:0,train_loss:0.008331758892116833,train_accuracy:0.8819095492362976
epoch:0,test_loss:0.01679145306656041,test_accuracy:0.5967926979064941
epoch:1,train_loss:0.005126319089028684,train_accuracy:0.998473048210144
epoch:1,test_loss:0.017573907612934963,test_accuracy:0.6139748096466064
epoch:2,train_loss:0.007197462297999843,train_accuracy:0.9156007766723633
epoch:2,test_loss:0.016734884640058973,test_accuracy:0.6185567378997803
epoch:3,train_loss:0.005085018022716584,train_accuracy:0.9987275004386902
epoch:3,test_loss:0.01743800629182097,test_accuracy:0.6300114989280701
epoch:4,train_loss:0.006601092374558969,train_accuracy:0.9376427531242371
epoch:4,test_loss:0.016479287087439398,test_accuracy:0.643757164478302
epoch:5,train_loss:0.00506871555700665,train_accuracy:0.9986003041267395
epoch:5,test_loss:0.017427262571669117,test_accuracy:0.6288660168647766
epoch:6,train_loss:0.0061765675704846265,train_accuracy:0.9571722745895386
epoch:6,test_loss:0.017250695315969497,t

In [13]:
#mobilevit50 Augmentation   0.592
from timm.models.mobilevit import mobilevitv2_050
model=mobilevitv2_050(img_size=(128,128),num_classes=10,pretrained=False).cuda()
train(model)

100%|██████████| 8732/8732 [00:19<00:00, 440.28it/s]


7859 0
7859


100%|██████████| 8731/8731 [00:18<00:00, 464.96it/s]


7858 0
7858


100%|██████████| 8732/8732 [00:12<00:00, 679.81it/s]


0 873
1116163
epoch:0,train_loss:0.022370786453270373,train_accuracy:0.2052685171365738
epoch:0,test_loss:0.02085889642044714,test_accuracy:0.3161512315273285
epoch:1,train_loss:0.018729370337191513,train_accuracy:0.36187809705734253
epoch:1,test_loss:0.01862672186389412,test_accuracy:0.4352806508541107
epoch:2,train_loss:0.01858326132044473,train_accuracy:0.38177651166915894
epoch:2,test_loss:0.01908537270959983,test_accuracy:0.4020618796348572
epoch:3,train_loss:0.015421238939397161,train_accuracy:0.5388725996017456
epoch:3,test_loss:0.017507382821108194,test_accuracy:0.4776632487773895
epoch:4,train_loss:0.01602031168788406,train_accuracy:0.49745482206344604
epoch:4,test_loss:0.018612483659287634,test_accuracy:0.40320736169815063
epoch:5,train_loss:0.012741484507332474,train_accuracy:0.6710777282714844
epoch:5,test_loss:0.017069286224085576,test_accuracy:0.544100821018219
epoch:6,train_loss:0.013974776095669213,train_accuracy:0.603970468044281
epoch:6,test_loss:0.018528264276213936,

In [14]:
#mobilevit50 Synthetic   0.62
from timm.models.mobilevit import mobilevitv2_050,MobileVitBlock
model=mobilevitv2_050(img_size=(128,128),num_classes=10,pretrained=False).cuda()
train_synthetic(model)

100%|██████████| 8732/8732 [00:20<00:00, 424.23it/s]


7859 0
7859


100%|██████████| 8493/8493 [00:10<00:00, 791.06it/s]


8493 0
8493


100%|██████████| 8732/8732 [00:15<00:00, 561.26it/s]


0 873
1116163
epoch:0,train_loss:0.020080738335437297,train_accuracy:0.3622983694076538
epoch:0,test_loss:0.021348708000925944,test_accuracy:0.30355098843574524
epoch:1,train_loss:0.01929993269355231,train_accuracy:0.3290494680404663
epoch:1,test_loss:0.01850224729256122,test_accuracy:0.444444477558136
epoch:2,train_loss:0.013317038534836593,train_accuracy:0.6548922657966614
epoch:2,test_loss:0.01828427727143243,test_accuracy:0.43184423446655273
epoch:3,train_loss:0.015639615128674053,train_accuracy:0.5112609267234802
epoch:3,test_loss:0.01585560101687157,test_accuracy:0.5463917851448059
epoch:4,train_loss:0.010034207382289734,train_accuracy:0.7884140014648438
epoch:4,test_loss:0.01758987838050344,test_accuracy:0.46620848774909973
epoch:5,train_loss:0.013007844233121532,train_accuracy:0.6538999676704407
epoch:5,test_loss:0.01553967537066098,test_accuracy:0.5761741399765015
epoch:6,train_loss:0.008404414343297742,train_accuracy:0.8539974093437195
epoch:6,test_loss:0.01670151764584571,te

In [8]:
#mobilevit50 Synthetic 2  0.62
from timm.models.mobilevit import mobilevitv2_050,MobileVitBlock
model=mobilevitv2_050(img_size=(128,128),num_classes=10,pretrained=False).cuda()
train_synthetic(model,root="./Synthetic_us8k_2") #0.628
train_synthetic(model,root="./Synthetic_us8k_3") #0.748
train_synthetic(model,root="./Synthetic_us8k_4") #0.745
train_synthetic(model,root="./Synthetic_us8k_5") #0.740

100%|██████████| 8732/8732 [00:27<00:00, 317.13it/s]


7859 0
7859


100%|██████████| 9061/9061 [00:12<00:00, 714.43it/s]


9061 0
9061


100%|██████████| 8732/8732 [00:22<00:00, 386.07it/s]


...

7859 0
7859


100%|██████████| 8756/8756 [00:11<00:00, 739.61it/s]


8756 0
8756


100%|██████████| 8732/8732 [00:20<00:00, 426.43it/s]


0 873
1116163
epoch:0,train_loss:0.008799605524229535,train_accuracy:0.8701462149620056
epoch:0,test_loss:0.015835410677417176,test_accuracy:0.6242840886116028
epoch:1,train_loss:0.005133163103818863,train_accuracy:0.9993637204170227
epoch:1,test_loss:0.01735665421305653,test_accuracy:0.6471936106681824
epoch:2,train_loss:0.007745126383987504,train_accuracy:0.9044084548950195
epoch:2,test_loss:0.01551602973697658,test_accuracy:0.6552119255065918
epoch:3,train_loss:0.005115834754119166,train_accuracy:0.9989820122718811
epoch:3,test_loss:0.017361229364541524,test_accuracy:0.6471936106681824
epoch:4,train_loss:0.007076577482129843,train_accuracy:0.9286203980445862
epoch:4,test_loss:0.016433965845752553,test_accuracy:0.612829327583313
epoch:5,train_loss:0.005092651239711375,train_accuracy:0.9998726844787598
epoch:5,test_loss:0.017328539801353987,test_accuracy:0.636884331703186
epoch:6,train_loss:0.006599415323406444,train_accuracy:0.947464644908905
epoch:6,test_loss:0.016577467885623684,te

In [17]:
#mobilevit150 Augmentation  0.683
from timm.models.mobilevit import mobilevitv2_150
model=mobilevitv2_150(img_size=(128,128),num_classes=10,pretrained=False).cuda()
train(model)

100%|██████████| 8732/8732 [00:35<00:00, 246.25it/s]


7859 0
7859


100%|██████████| 8731/8731 [00:35<00:00, 248.82it/s]


7858 0
7858


100%|██████████| 8732/8732 [00:12<00:00, 696.72it/s]


0 873
9833443
epoch:0,train_loss:0.020097700473642677,train_accuracy:0.3189106583595276
epoch:0,test_loss:0.019670403290450368,test_accuracy:0.34593358635902405
epoch:1,train_loss:0.014044099067030767,train_accuracy:0.6080926060676575
epoch:1,test_loss:0.01618742901844667,test_accuracy:0.605956494808197
epoch:2,train_loss:0.014187973657313361,train_accuracy:0.599134624004364
epoch:2,test_loss:0.017182797637318965,test_accuracy:0.5257732272148132
epoch:3,train_loss:0.008950285237168277,train_accuracy:0.8420917987823486
epoch:3,test_loss:0.015334113220442066,test_accuracy:0.6254295706748962
epoch:4,train_loss:0.010170714458942291,train_accuracy:0.7865868806838989
epoch:4,test_loss:0.018428420283130763,test_accuracy:0.5315005779266357
epoch:5,train_loss:0.006711718591361696,train_accuracy:0.9423590302467346
epoch:5,test_loss:0.01642734392675201,test_accuracy:0.6162657737731934
epoch:6,train_loss:0.007075391089712946,train_accuracy:0.9258080720901489
epoch:6,test_loss:0.0200696522700691,te

In [16]:
#mobilevit150 Synthetic   0.746
from timm.models.mobilevit import mobilevitv2_150
model=mobilevitv2_150(img_size=(128,128),num_classes=10,pretrained=False).cuda()
train_synthetic(model)

100%|██████████| 8732/8732 [00:19<00:00, 437.10it/s]


7859 0
7859


100%|██████████| 8493/8493 [00:09<00:00, 868.44it/s] 


8493 0
8493


100%|██████████| 8732/8732 [00:13<00:00, 644.55it/s]


0 873
9833443
epoch:0,train_loss:0.01399577525792492,train_accuracy:0.6262804865837097
epoch:0,test_loss:0.017533924025222063,test_accuracy:0.48224514722824097
epoch:1,train_loss:0.014179669501657385,train_accuracy:0.6040208339691162
epoch:1,test_loss:0.015867094813343585,test_accuracy:0.5899198651313782
epoch:2,train_loss:0.0075669839344835385,train_accuracy:0.8973272442817688
epoch:2,test_loss:0.01519788799963183,test_accuracy:0.5922107696533203
epoch:3,train_loss:0.008877085894630647,train_accuracy:0.8451456427574158
epoch:3,test_loss:0.016265523665822523,test_accuracy:0.5601375102996826
epoch:4,train_loss:0.006296598587049607,train_accuracy:0.9534911513328552
epoch:4,test_loss:0.017671565146517236,test_accuracy:0.5521191358566284
epoch:5,train_loss:0.006530352351106992,train_accuracy:0.9473214745521545
epoch:5,test_loss:0.01629941255912475,test_accuracy:0.5830469727516174
epoch:6,train_loss:0.005681878024440035,train_accuracy:0.9812787175178528
epoch:6,test_loss:0.0168956421221791,

In [9]:
#mobilevit150 Synthetic 2  0.746
from timm.models.mobilevit import mobilevitv2_150
model=mobilevitv2_150(img_size=(128,128),num_classes=10,pretrained=False).cuda()
train_synthetic(model,root="./Synthetic_us8k_2") #0.735
train_synthetic(model,root="./Synthetic_us8k_3") #0.735
train_synthetic(model,root="./Synthetic_us8k_4") #0.764
train_synthetic(model,root="./Synthetic_us8k_5") #0.726

100%|██████████| 8732/8732 [00:26<00:00, 327.59it/s]


7859 0
7859


100%|██████████| 9061/9061 [00:12<00:00, 703.04it/s]


9061 0
9061


100%|██████████| 8732/8732 [00:21<00:00, 406.34it/s]


...

7859 0
7859


100%|██████████| 8756/8756 [00:07<00:00, 1181.97it/s]


8756 0
8756


100%|██████████| 8732/8732 [00:12<00:00, 697.47it/s]


0 873
9833443
epoch:0,train_loss:0.007672591642899947,train_accuracy:0.9079488515853882
epoch:0,test_loss:0.013418837647804006,test_accuracy:0.7136312127113342
epoch:1,train_loss:0.00511833576016002,train_accuracy:0.997964084148407
epoch:1,test_loss:0.014911855345875673,test_accuracy:0.7262313961982727
epoch:2,train_loss:0.006588470634939247,train_accuracy:0.9452946782112122
epoch:2,test_loss:0.013434616697339804,test_accuracy:0.7159221172332764
epoch:3,train_loss:0.005084693143652632,train_accuracy:0.9996182322502136
epoch:3,test_loss:0.014659678130351936,test_accuracy:0.7159221172332764
epoch:4,train_loss:0.005989110363624367,train_accuracy:0.9668799042701721
epoch:4,test_loss:0.014086255775967277,test_accuracy:0.70561283826828
epoch:5,train_loss:0.005060892468236575,train_accuracy:0.9997454881668091
epoch:5,test_loss:0.014638194240382174,test_accuracy:0.7170675992965698
epoch:6,train_loss:0.005651804023324871,train_accuracy:0.9800137281417847
epoch:6,test_loss:0.014211193938981763,t

In [18]:
#covnext tiny Augmentation   0.6
from timm.models.convnext import convnext_tiny
model=convnext_tiny(num_classes=10,pretrained=False).cuda()
train(model)

100%|██████████| 8732/8732 [00:17<00:00, 509.87it/s]


7859 0
7859


100%|██████████| 8731/8731 [00:17<00:00, 497.46it/s]


7858 0
7858


100%|██████████| 8732/8732 [00:12<00:00, 707.44it/s]


0 873
27827818
epoch:0,train_loss:0.022027743897325182,train_accuracy:0.1913972944021225
epoch:0,test_loss:0.02123073846067623,test_accuracy:0.2737686336040497
epoch:1,train_loss:0.01881964473054524,train_accuracy:0.3667133152484894
epoch:1,test_loss:0.019477361392865614,test_accuracy:0.4306987524032593
epoch:2,train_loss:0.019054055835916118,train_accuracy:0.3625604510307312
epoch:2,test_loss:0.019186743619379172,test_accuracy:0.46849945187568665
epoch:3,train_loss:0.015647368722933368,train_accuracy:0.5267845392227173
epoch:3,test_loss:0.018432563932490924,test_accuracy:0.5922107696533203
epoch:4,train_loss:0.0172617684705593,train_accuracy:0.45469585061073303
epoch:4,test_loss:0.019333423071573964,test_accuracy:0.4490263760089874
epoch:5,train_loss:0.013873976944572583,train_accuracy:0.6214531064033508
epoch:5,test_loss:0.018551641723134674,test_accuracy:0.4833906292915344
epoch:6,train_loss:0.016391940504031862,train_accuracy:0.49478238821029663
epoch:6,test_loss:0.0190751295865333

In [19]:
#covnext tiny Synthetic  0.658
model=convnext_tiny(num_classes=10,pretrained=False).cuda()
train_synthetic(model)

100%|██████████| 8732/8732 [00:28<00:00, 301.91it/s]


7859 0
7859


100%|██████████| 8493/8493 [00:36<00:00, 233.27it/s]


8493 0
8493


100%|██████████| 8732/8732 [00:26<00:00, 329.57it/s]


0 873
27827818
epoch:0,train_loss:0.018749012734294006,train_accuracy:0.35676440596580505
epoch:0,test_loss:0.02081209072952008,test_accuracy:0.36311569809913635
epoch:1,train_loss:0.018041006859545034,train_accuracy:0.4009415805339813
epoch:1,test_loss:0.01922077070260239,test_accuracy:0.5383734703063965
epoch:2,train_loss:0.012187641128471318,train_accuracy:0.6876251101493835
epoch:2,test_loss:0.020241917203791773,test_accuracy:0.4410080313682556
epoch:3,train_loss:0.015287593767409076,train_accuracy:0.5541417002677917
epoch:3,test_loss:0.01761249428079437,test_accuracy:0.601374626159668
epoch:4,train_loss:0.01034638431262285,train_accuracy:0.7879430651664734
epoch:4,test_loss:0.01850200122710902,test_accuracy:0.5555555820465088
epoch:5,train_loss:0.013788335998939003,train_accuracy:0.6256520748138428
epoch:5,test_loss:0.017569997329755612,test_accuracy:0.5257732272148132
epoch:6,train_loss:0.009631041466775722,train_accuracy:0.8141999840736389
epoch:6,test_loss:0.019357092055531575,

In [10]:
#covnext tiny Synthetic 2 0.658
from timm.models.convnext import convnext_tiny
model=convnext_tiny(num_classes=10,pretrained=False).cuda()
train_synthetic(model,root="./Synthetic_us8k_2") #0.623
train_synthetic(model,root="./Synthetic_us8k_3") #0.620
train_synthetic(model,root="./Synthetic_us8k_4") #0.610
train_synthetic(model,root="./Synthetic_us8k_5") #0.628

100%|██████████| 8732/8732 [00:17<00:00, 498.54it/s]


7859 0
7859


100%|██████████| 9061/9061 [00:07<00:00, 1154.63it/s]


9061 0
9061


100%|██████████| 8732/8732 [00:12<00:00, 689.62it/s]


...

7859 0
7859


100%|██████████| 8756/8756 [00:07<00:00, 1143.94it/s]


8756 0
8756


100%|██████████| 8732/8732 [00:12<00:00, 684.58it/s]


0 873
27827818
epoch:0,train_loss:0.00784736540204262,train_accuracy:0.8980128169059753
epoch:0,test_loss:0.015554841441387163,test_accuracy:0.6162657737731934
epoch:1,train_loss:0.00509442839705621,train_accuracy:0.9996182322502136
epoch:1,test_loss:0.017806504736787812,test_accuracy:0.6288660168647766
epoch:2,train_loss:0.0066548668532243,train_accuracy:0.9393559098243713
epoch:2,test_loss:0.016621821251658366,test_accuracy:0.6162657737731934
epoch:3,train_loss:0.005061489091087562,train_accuracy:0.9998726844787598
epoch:3,test_loss:0.018395300583331445,test_accuracy:0.6105384230613708
epoch:4,train_loss:0.005965148098341348,train_accuracy:0.9685930013656616
epoch:4,test_loss:0.018053554723899117,test_accuracy:0.612829327583313
epoch:5,train_loss:0.00504639349239809,train_accuracy:0.9997454881668091
epoch:5,test_loss:0.01857894338146791,test_accuracy:0.6093929409980774
epoch:6,train_loss:0.005567705470604023,train_accuracy:0.9828689098358154
epoch:6,test_loss:0.018275447590927352,tes

In [2]:
#deit Augmentation  0.674
from timm.models.deit import deit3_base_patch16_224
model=deit3_base_patch16_224(num_classes=10,pretrained=False,img_size=(128,128)).cuda()
train(model)

100%|██████████| 8732/8732 [00:19<00:00, 440.75it/s]


7859 0
7859


100%|██████████| 8731/8731 [00:25<00:00, 343.05it/s]


7858 0
7858


100%|██████████| 8732/8732 [00:20<00:00, 436.10it/s]


0 873
85722634
epoch:0,train_loss:0.022348851155130335,train_accuracy:0.1880885660648346
epoch:0,test_loss:0.021428264430025327,test_accuracy:0.253150075674057
epoch:1,train_loss:0.019154020811343774,train_accuracy:0.3401196002960205
epoch:1,test_loss:0.018637596946401696,test_accuracy:0.4845361113548279
epoch:2,train_loss:0.01880120841198885,train_accuracy:0.379231333732605
epoch:2,test_loss:0.017341153318529687,test_accuracy:0.42611685395240784
epoch:3,train_loss:0.014206205619960542,train_accuracy:0.5982949137687683
epoch:3,test_loss:0.016072125642433474,test_accuracy:0.5979381799697876
epoch:4,train_loss:0.01568195746429824,train_accuracy:0.5264698266983032
epoch:4,test_loss:0.01691645745695798,test_accuracy:0.46277207136154175
epoch:5,train_loss:0.011461457324583183,train_accuracy:0.7334266304969788
epoch:5,test_loss:0.015768063846732332,test_accuracy:0.5945017337799072
epoch:6,train_loss:0.014065691231771048,train_accuracy:0.599134624004364
epoch:6,test_loss:0.01616216320352456,t

In [3]:
#deit Synthetic  0.722
from timm.models.deit import deit3_base_patch16_224
model=deit3_base_patch16_224(num_classes=10,pretrained=False,img_size=(128,128)).cuda()
train_synthetic(model)

100%|██████████| 8732/8732 [00:20<00:00, 429.69it/s]


7859 0
7859


100%|██████████| 8493/8493 [00:13<00:00, 645.33it/s]


8493 0
8493


100%|██████████| 8732/8732 [00:22<00:00, 392.42it/s]


0 873
85722634
epoch:0,train_loss:0.01795931565920175,train_accuracy:0.4032732844352722
epoch:0,test_loss:0.020785683072582824,test_accuracy:0.3734249770641327
epoch:1,train_loss:0.016299657776756495,train_accuracy:0.49535560607910156
epoch:1,test_loss:0.01611103484447606,test_accuracy:0.5647193789482117
epoch:2,train_loss:0.009650437315066459,train_accuracy:0.8051336407661438
epoch:2,test_loss:0.01824239521092156,test_accuracy:0.5761741399765015
epoch:3,train_loss:0.012309324409173242,train_accuracy:0.6817660927772522
epoch:3,test_loss:0.015510416112814573,test_accuracy:0.5945017337799072
epoch:4,train_loss:0.007931485651070809,train_accuracy:0.8828447461128235
epoch:4,test_loss:0.01596667717412575,test_accuracy:0.5967926979064941
epoch:5,train_loss:0.009603683442495481,train_accuracy:0.8130804896354675
epoch:5,test_loss:0.016720817807340676,test_accuracy:0.5922107696533203
epoch:6,train_loss:0.007089674781604831,train_accuracy:0.9175792336463928
epoch:6,test_loss:0.015044792001599708

In [11]:
#deit Synthetic 2 0.722
from timm.models.deit import deit3_base_patch16_224
model=deit3_base_patch16_224(num_classes=10,pretrained=False,img_size=(128,128)).cuda()
train_synthetic(model,root="./Synthetic_us8k_2") #0.728
train_synthetic(model,root="./Synthetic_us8k_3") #0.713
train_synthetic(model,root="./Synthetic_us8k_4") #0.692
train_synthetic(model,root="./Synthetic_us8k_5") #0.678

100%|██████████| 8732/8732 [00:17<00:00, 497.64it/s]


7859 0
7859


100%|██████████| 9061/9061 [00:07<00:00, 1136.70it/s]


9061 0
9061


100%|██████████| 8732/8732 [00:12<00:00, 685.67it/s]


...

7859 0
7859


100%|██████████| 8756/8756 [00:28<00:00, 302.72it/s]


8756 0
8756


100%|██████████| 8732/8732 [00:23<00:00, 375.43it/s]


0 873
85722634
epoch:0,train_loss:0.008173917235986128,train_accuracy:0.8828232288360596
epoch:0,test_loss:0.015717513919013198,test_accuracy:0.6540664434432983
epoch:1,train_loss:0.005095106641638596,train_accuracy:0.9989820122718811
epoch:1,test_loss:0.01700918903875187,test_accuracy:0.6609393358230591
epoch:2,train_loss:0.00698323026401586,train_accuracy:0.9240521192550659
epoch:2,test_loss:0.01640461236750547,test_accuracy:0.6517755389213562
epoch:3,train_loss:0.005053436199449859,train_accuracy:0.9997454881668091
epoch:3,test_loss:0.017077956276114727,test_accuracy:0.6471936106681824
epoch:4,train_loss:0.006172586421326123,train_accuracy:0.9559159874916077
epoch:4,test_loss:0.016430570634096895,test_accuracy:0.643757164478302
epoch:5,train_loss:0.005054019769801334,train_accuracy:0.9994909763336182
epoch:5,test_loss:0.016796129539113958,test_accuracy:0.663230299949646
epoch:6,train_loss:0.005700353306687112,train_accuracy:0.9756738543510437
epoch:6,test_loss:0.01705534146823424,te

In [1]:
#Judge Model Training
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm.models as models
import numpy as np
import pandas as pd
from utils import US8K
from timm.models.xception import xception
model=xception(num_classes=10,pretrained=False).cuda()
loss_fn=nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer=torch.optim.AdamW(model.parameters(),lr=0.0001)
train_dataset=US8K(train=True,transform_size=128,all_in=True)
print(len(train_dataset.data))
test_dataset=US8K(train=False,transform_size=128)
epoches=500
print(sum(p.numel() for p in model.parameters() if p.requires_grad))
for epoch in range(epoches):
    model.train()
    train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=100,shuffle=True)
    test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=100,shuffle=True)
    accuracy=0
    losses_all=0
    all_have=0
    for data,label in train_loader:
        data=data.cuda()
        label=label.cuda()
        output=model(data)
        loss=loss_fn(output,label)
        accuracy+=torch.sum(torch.argmax(output,dim=1)==torch.argmax(label,dim=1))
        all_have+=data.shape[0]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses_all+=loss.item()
    print('epoch:{},train_loss:{},train_accuracy:{}'.format(epoch,losses_all/all_have,accuracy/all_have)) 

    model.eval()
    all_have=0

    accuracy=0
    losses_all=0
    for data,label in test_loader:
        data=data.cuda()
        label=label.cuda()
        output=model(data)
        loss=loss_fn(output,label)
        all_have+=data.shape[0]
        accuracy+=torch.sum(torch.argmax(output,dim=1)==torch.argmax(label,dim=1))
        losses_all+=loss.item()
    print('epoch:{},test_loss:{},test_accuracy:{}'.format(epoch,losses_all/all_have,accuracy/all_have))
torch.save(model,"judge.pt")

ModuleNotFoundError: No module named 'utils'

In [8]:
#Judeg Model
torch.save(model,"judge.pt")